# 分类 Moon Data

在此 notebook 中，你的任务是在 SageMaker 中构建和部署一个**自定义模型**。你将定义和训练一个自定义 PyTorch 神经网络，并创建一个二元分类器，它会将数据分成两个类别；数据看起来像两个月亮，通常称为 **moon data**。

该 notebook 将分成以下几个步骤：
* 生成 moon data
* 将数据加载到 S3 存储桶中
* 定义一个 PyTorch 二元分类器
* 完成训练脚本
* 训练和部署自定义模型
* 评估模型的效果

训练和部署自定义模型是很实用的技能。尤其是遇到 LinearLearner 等传统算法无法解决的问题时，自定义模型就很有用。

---

在下面加载必要的库。

In [ ]:
# data 
import pandas as pd 
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

%matplotlib inline

## 生成 Moon Data

我在下面编写了生成 Moon Data 的代码，并使用了 sklearn 的 [make_moons](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html) 和 [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)。

我指定了数据点的数量以及噪点参数，然后显示生成的数据。

In [ ]:
# set data params
np.random.seed(0)
num_pts = 600
noise_val = 0.25

# generate data
# X = 2D points, Y = class labels (0 or 1)
X, Y = make_moons(num_pts, noise=noise_val)

# Split into test and training data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
    test_size=0.25, random_state=1)

In [ ]:
# plot
# points are colored by class, Y_train
# 0 labels = purple, 1 = yellow
plt.figure(figsize=(8,5))
plt.scatter(X_train[:,0], X_train[:,1], c=Y_train)
plt.title('Moon Data')
plt.show()

## SageMaker 资源

以下单元格存储了 SageMaker 会话和角色（用于创建评估器和模型），并创建默认的 S3 存储桶。创建此存储桶后，你可以将所有本地数据上传到 S3。

In [ ]:
# sagemaker
import boto3
import sagemaker
from sagemaker import get_execution_role

In [ ]:
# SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# default S3 bucket
bucket = sagemaker_session.default_bucket()

### 练习：创建 csv 文件

定义一个函数，它会接受 x（特征）和 y（标签），并将它们保存到一个 `.csv` 文件中，文件路径为 `data_dir/filename`。SageMaker 要求 `.csv` 文件是特定的格式（请参阅[此文档](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-training.html)）：
> Amazon SageMaker 要求  CSV 文件没有标题，并且目标变量在第一列。

建议使用 pandas 将特征和标签合并成一个 DataFrame，然后将其转换成 `.csv` 文件。在创建 `.csv` 文件时，记得设置 `header=False` 和 `index=False`，这样就不会在 `.csv` 文件中包含任何多余的信息，例如列名。

In [ ]:
import os

def make_csv(x, y, filename, data_dir):
    '''Merges features and labels and converts them into one csv file with labels in the first column.
       :param x: Data features
       :param y: Data labels
       :param file_name: Name of csv file, ex. 'train.csv'
       :param data_dir: The directory where files will be saved
       '''
    # make data dir, if it does not exist
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    # your code here
    
    
    # nothing is returned, but a print statement indicates that the function has run
    print('Path created: '+str(data_dir)+'/'+str(filename))

以下单元格将运行上述函数，并在指定目录创建 `train.csv` 文件。

In [ ]:
data_dir = 'data_moon' # the folder we will use for storing data
name = 'train.csv'

# create 'train.csv'
make_csv(X_train, Y_train, name, data_dir)

### 将数据上传到 S3

使用 `sagemaker_session.upload_data` 将在本地的 `train.csv` 文件上传到 S3。这个函数需要知道：数据在本地的存储位置，以及将数据上传到 S3 的位置（存储桶和 prefix）。

In [ ]:
# specify where to upload in S3
prefix = 'sagemaker/moon-data'

# upload to S3
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
print(input_data)

通过输出默认存储桶的内容，检查是否上传了数据。

In [ ]:
# iterate through S3 objects and print contents
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
     print(obj.key)

---
# 建模

上传了训练数据后，下面定义并训练模型。

在此 notebook 中，你将定义和训练一个**自定义 PyTorch 模型**，它是一个执行二元分类的神经网络。

### 练习：在 `model.py` 中定义模型

为了实现自定义分类器，首先需要定义一个神经网络。我在 `source` 目录中已经提供了一些起始代码，你可以在其中找到文件 `model.py`。你需要完成类 `SimpleNet`；指定神经网络的层级及其前馈行为。建议参阅[三层 MLP 的代码](https://github.com/udacity/deep-learning-v2-pytorch/blob/master/convolutional-neural-networks/mnist-mlp/mnist_mlp_solution.ipynb)。

此模型应该：
* 接受 `input_dim` 个特征
* 创建指定大小的线性隐藏层
* 返回**一个输出值**，表示类别分数

返回的输出值应该是 [S 型函数激活过的](https://pytorch.org/docs/stable/nn.html#sigmoid)类别分数，它是一个 0-1 之间的值，可以四舍五入为预测类别标签。

你可以在下面使用 !pygmentize 显示 `model.py` 文件中的代码。请通读代码；所有任务都标有 TODO 注释。请转到该文件并完成任务来定义 `SimpleNet`。

In [ ]:
!pygmentize source/model.py

## 训练脚本

为了实现自定义分类器，你还需要完成 `train.py` 脚本。你可以在 `source` 目录下找到该脚本。

典型的训练脚本会：

* 从指定目录加载训练数据
* 解析任何训练和模型超参数（例如神经网络中的节点数，训练周期数等）
* 根据指定的参数实例化你设计的模型
* 训练该模型
* 最后保存该模型，以便稍后托管/部署该模型

### 练习：完成 `train.py` 脚本

我们已经向你提供了大部分训练脚本的代码。几乎所有任务都位于 __name__ == '__main__': 部分。要完成 `train.py` 文件，你需要：

* 使用 `parser.add_argument` 定义任何其他模型训练超参数
* 在 if __name__ == '__main__': 部分定义模型
* 在此部分训练模型

你可以在下面使用 !pygmentize 显示现有 train.py 文件。请通读代码；所有任务都标有 TODO 注释。

In [ ]:
!pygmentize source/train.py

### 练习：创建 PyTorch 评估器

你已经练习了在 SageMaker 中实例化内置模型。所有评估器都需要传入一些构造函数参数。在 SageMaker 中构建自定义模型时，必须指定**入口点**。入口点是模型被训练时将执行的训练脚本，即你在上面指定的 `train.py` 函数。

看看你能否仅通过参考 [PyTorch 评估器文档](https://sagemaker.readthedocs.io/en/stable/sagemaker.pytorch.html)完成此任务，即实例化一个 PyTorch 评估器。建议将可选参数 `framework_version` 设为 PyTorch 的**最新版本**。

#### 实例类型

建议使用免费套餐里提供的实例 `'ml.c4.xlarge'` 训练模型，并使用 `'ml.t2.medium'` 部署模型。

In [ ]:
# import a PyTorch wrapper
from sagemaker.pytorch import PyTorch

# specify an output path
output_path = None

# instantiate a pytorch estimator
estimator = None


## 训练评估器

实例化评估器后，通过调用 `.fit()` 训练该评估器。`train.py` 文件会明确加载 `.csv` 文件，所以你不需要将输入数据转换成任何其他格式。

In [ ]:
%%time 
# train the estimator on S3 training data
estimator.fit({'train': input_data})

## 创建训练过的模型

PyTorch 模型并没有自动带 `.predict()` 函数（而很多 Scikit-learn 模型都内置了这些函数），你可能已经发现了，我们已经提供了一个 `predict.py` 文件。此文件负责加载训练过的模型并将其应用到传入的 NumPy 数据上。在创建 PyTorch 评估器时，你指定了训练脚本 `train.py` 的位置。

> 如何告诉 PyTorch 模型 `predict.py` 文件在哪？

在部署自定义 PyTorch 模型之前，你需要先创建 `PyTorchModel`。在之前的练习中，你发现调用 `.deploy()` 会同时创建一个**模型**和**端点**，但是对于 PyTorch 模型，这些步骤需要分开来完成。

### 练习：实例化 `PyTorchModel`

你可以根据训练过的评估器属性创建一个 `PyTorchModel`（与 PyTorch 评估器不同）。此模型负责知道如何执行特定的 `predict.py` 脚本。你将部署此模型来创建端点。

#### 模型参数

要实例化 `PyTorchModel`（[参考文档](https://sagemaker.readthedocs.io/en/stable/sagemaker.pytorch.html#sagemaker.pytorch.model.PyTorchModel)），你需要传入和 PyTorch 评估器一样的参数，并添加或修改一些参数：
* **model_data**：评估器创建的已训练 `model.tar.gz` 文件，可以作为 `estimator.model_data` 访问。
* **entry_point**：这次，它是 SageMaker 运行的预测 Python 脚本 `predict.py`（而不是训练脚本）的路径。


In [ ]:
%%time
# importing PyTorchModel
from sagemaker.pytorch import PyTorchModel

# Create a model from the trained estimator data
# And point to the prediction script
model = None


### 练习：部署训练过的模型

部署模型以创建预测器。我们将使用预测器对测试数据做出预测并评估模型。

In [ ]:
%%time
# deploy and create a predictor
predictor = None

---
## 评估模型

部署模型后，你可以将模型应用到测试数据上，看看模型的性能。

下面提供的函数会接受部署的预测器、一些测试特征和标签，并返回指标字典，计算假负例和假正例，以及召回率、精确率和准确率。

In [ ]:
# code to evaluate the endpoint on test data
# returns a variety of model metrics
def evaluate(predictor, test_features, test_labels, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  
    Return binary classification metrics.
    :param predictor: A prediction endpoint
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all performance metrics
    :return: A dictionary of performance metrics.
    """
    
    # rounding and squeezing array
    test_preds = np.squeeze(np.round(predictor.predict(test_features)))
    
    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    # print metrics
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actuals'], colnames=['predictions']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print()
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 
            'Precision': precision, 'Recall': recall, 'Accuracy': accuracy}



### 测试结果

以下单元格将运行 `evaluate` 函数。

代码假设你已经在前面运行的单元格中定义了 `predictor`、`X_test` 和 `Y_test`。

In [ ]:
# get metrics for custom predictor
metrics = evaluate(predictor, X_test, Y_test, True)

## 删除端点

最后，我添加了删除预测端点的便利函数。如果你已经评估完模型，应该删除模型端点！

In [ ]:
# Accepts a predictor endpoint as input
# And deletes the endpoint by name
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

In [ ]:
# delete the predictor endpoint 
delete_endpoint(predictor)

## 最终清理！

* 仔细检查是否删除了所有端点。
* 还建议直接通过 AWS 控制台手动删除 S3 存储桶、模型和端点配置。

[这篇文档](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html)提供了彻底清理的说明。

---
# 总结

在此 notebook 中，你学习了如何在 SageMaker 中训练和部署自定义 PyTorch 模型。SageMaker 有很多内置模型，这些模型适合常见的聚类和分类任务，但是了解如何创建自定义深度学习模型很有用，便于灵活地从各种数据中学习规律。